# Visualizing and Analyzing Global Land Cover Data

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geemap/blob/master/examples/workshops/GEE_Workshop_2022_Part2.ipynb)

## Import libraries

In [ ]:
# pip install geemap

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

## Visualizing Global Land Cover Data Products

### Creating Dynamic World Land Cover Composites

- App: https://www.dynamicworld.app
- App2: https://earthoutreach.users.earthengine.app/view/dynamicworld
- Paper: https://doi.org/10.1038/s41597-022-01307-4
- Model: https://github.com/google/dynamicworld
- Training data: https://doi.pangaea.de/10.1594/PANGAEA.933475
- Data: https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1
- JavaScript tutorial: https://developers.google.com/earth-engine/tutorials/community/introduction-to-dynamic-world-pt-1

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map

In [ ]:
# Set the region of interest by simply drawing a polygon on the map
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)
    Map.addLayer(region, {}, "Region")

Map.centerObject(region)

In [ ]:
# Set the date range
start_date = "2021-01-01"
end_date = "2022-01-01"

In [ ]:
# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(region, start_date, end_date, clip=True)
vis_params = {"bands": ["B8", "B4", "B3"], "min": 0, "max": 3000}
Map.addLayer(image, vis_params, "Sentinel-2 image")

In [ ]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type="class"
)

dwVisParams = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

Map.addLayer(landcover, dwVisParams, "Land Cover Class")
Map

In [ ]:
Map.add_legend(title="Dynamic World Land Cover", builtin_legend="Dynamic_World")

In [ ]:
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type="visualize"
)
Map.addLayer(landcover, {}, "Land Cover Visualize")

In [ ]:
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type="probability"
)
Map.addLayer(landcover, {}, "Land Cover Probability")

In [ ]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(
    region, start_date, end_date, clip=True, return_type="hillshade"
)
Map.addLayer(landcover, {}, "Land Cover")

![](https://i.imgur.com/GEzsSii.png)

### Comparing Global Land Cover Data Products

Visualizing [ESA Global Land Cover](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200).

In [ ]:
start_date = "2021-01-01"
end_date = "2022-01-01"
region = ee.Geometry.BBox(-179, -89, 179, 89)

In [ ]:
Map = geemap.Map()

esa = ee.ImageCollection("ESA/WorldCover/v200").first()
esa_vis = {"bands": ["Map"]}

Map.addLayer(esa, esa_vis, "ESA LC 2021")
Map.add_legend(title="ESA Land Cover", builtin_legend="ESA_WorldCover")

Map

Visualizing [ESRI Global Land Cover](https://gee-community-catalog.org/projects/S2TSLULC/).

In [ ]:
Map = geemap.Map()

esri = (
    ee.ImageCollection(
        "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
    )
    .filterDate(start_date, end_date)
    .mosaic()
)
esri_vis = {
    "min": 1,
    "max": 11,
    "palette": [
        "#1A5BAB",
        "#358221",
        "#000000",
        "#87D19E",
        "#FFDB5C",
        "#000000",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8",
        "#C6AD8D",
    ],
}

Map.addLayer(esri, esri_vis, "ESRI LC 2021")
Map.add_legend(title="ESRI Land Cover", builtin_legend="ESRI_LandCover_TS")

Map

In [ ]:
Map = geemap.Map()

dw_class = geemap.dynamic_world(region, start_date, end_date, return_type="class")
dw = geemap.dynamic_world(region, start_date, end_date, return_type="hillshade")

dw_vis = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

Map.addLayer(dw_class, dw_vis, "DW LC 2021", False)
Map.addLayer(dw, {}, "DW LC Hillshade")

Map.add_legend(title="Dynamic World Land Cover", builtin_legend="Dynamic_World")
Map.setCenter(-88.9088, 43.0006, 12)
Map

Comparing Dynamic World and ESA Land Cover.

In [ ]:
Map = geemap.Map(center=[39.3322, -106.7349], zoom=10)

left_layer = geemap.ee_tile_layer(esa, esa_vis, "ESA LC")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World LC")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="ESA Land Cover", builtin_legend="ESA_WorldCover", position="bottomleft"
)
Map.add_legend(
    title="Dynamic World Land Cover",
    builtin_legend="Dynamic_World",
    position="bottomright",
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

Comparing Dynamic World with ESRI Land Cover.

In [ ]:
Map = geemap.Map(center=[-89.3998, 43.0886], zoom=10)

left_layer = geemap.ee_tile_layer(esri, esri_vis, "ESRI LC")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World LC")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="ESRI Land Cover", builtin_legend="ESRI_LandCover_TS", position="bottomleft"
)
Map.add_legend(
    title="Dynamic World Land Cover",
    builtin_legend="Dynamic_World",
    position="bottomright",
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

### Creating Dynamic World Time Series

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map

In [ ]:
# Set the region of interest by simply drawing a polygon on the map
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)
    Map.addLayer(region, {}, "Region")

Map.centerObject(region)

In [ ]:
# Set the date range
start_date = "2017-01-01"
end_date = "2023-01-01"

In [ ]:
images = geemap.dynamic_world_timeseries(
    region, start_date, end_date, frequency="year", return_type="class"
)

In [ ]:
vis_params = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}
Map.addLayer(images.first(), vis_params, "First image")
Map.add_legend(title="Dynamic World Land Cover", builtin_legend="Dynamic_World")
Map

In [ ]:
Map.ts_inspector(images, left_vis=vis_params, date_format="YYYY")

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.centerObject(region)

images = geemap.dynamic_world_timeseries(
    region, start_date, end_date, frequency="year", return_type="hillshade"
)
Map.ts_inspector(images, date_format="YYYY")
Map.add_legend(title="Dynamic World Land Cover", builtin_legend="Dynamic_World")

Map

![](https://i.imgur.com/5DGOuTC.png)

## Analyzing Global Land Cover Data

In [ ]:
Map = geemap.Map()
dataset = ee.ImageCollection("ESA/WorldCover/v200").first()
Map.addLayer(dataset, {"bands": ["Map"]}, "ESA Land Cover")
Map.add_legend(title="ESA Land Cover Type", builtin_legend="ESA_WorldCover")
Map

In [ ]:
df = geemap.image_area_by_group(
    dataset, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
df

In [ ]:
df.to_csv("esa2021.csv")

In [ ]:
esa_dict = {
    "10 Trees": "006400",
    "20 Shrubland": "ffbb22",
    "30 Grassland": "ffff4c",
    "40 Cropland": "f096ff",
    "50 Built-up": "fa0000",
    "60 Barren / sparse vegetation": "b4b4b4",
    "70 Snow and ice": "f0f0f0",
    "80 Open water": "0064c8",
    "90 Herbaceous wetland": "0096a0",
    "95 Mangroves": "00cf75",
    "100 Moss and lichen": "fae6a0",
}
classes = list(esa_dict.keys())
classes

In [ ]:
df["class"] = classes

In [ ]:
geemap.bar_chart(
    df,
    x="class",
    y="area",
    x_label="Land Cover Type",
    y_label="Area (km2)",
)

In [ ]:
geemap.pie_chart(df, names="class", values="area", height=500)

In [ ]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))
Map.addLayer(countries, {}, "Countries")
Map

In [ ]:
geemap.zonal_stats_by_group(
    dataset,
    countries,
    "esa_2021_country.csv",
    stat_type="SUM",
    denominator=1e6,
    scale=1000,
)

In [ ]:
geemap.bar_chart(
    "esa_2021_country.csv",
    x="NAME",
    y="Class_10",
    max_rows=30,
    x_label="Country",
    y_label="Forest Area (km2)",
)

In [ ]:
geemap.bar_chart(
    "esa_2021_country.csv",
    x="NAME",
    y="Class_40",
    max_rows=30,
    x_label="Country",
    y_label="Cropland Area (km2)",
)

In [ ]:
geemap.bar_chart(
    "esa_2021_country.csv",
    x="NAME",
    y=["Class_10", "Class_20", "Class_30", "Class_40"],
    max_rows=10,
    x_label="Country",
)

In [ ]:
geemap.pie_chart(
    "esa_2021_country.csv", names="NAME", values="Class_10", max_rows=30, height=500
)

In [ ]:
geemap.pie_chart(
    "esa_2021_country.csv", names="NAME", values="Class_40", max_rows=30, height=500
)

## Forest cover change analysis

https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2021_v1_9

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

In [ ]:
dataset = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

In [ ]:
dataset.bandNames().getInfo()

In [ ]:
first_bands = ["first_b50", "first_b40", "first_b30"]
first_image = dataset.select(first_bands)
Map.addLayer(first_image, {"bands": first_bands, "gamma": 1.5}, "Year 2000 Bands 5/4/3")
Map

In [ ]:
last_bands = ["last_b50", "last_b40", "last_b30"]
last_image = dataset.select(last_bands)
Map.addLayer(last_image, {"bands": last_bands, "gamma": 1.5}, "Year 2021 Bands 5/4/3")

In [ ]:
treecover = dataset.select(["treecover2000"])

treeCoverVisParam = {"min": 0, "max": 100, "palette": ["black", "green"]}

name1 = "Tree cover (%)"
Map.addLayer(treecover, treeCoverVisParam, name1)
Map.add_colorbar(treeCoverVisParam, label=name1, layer_name=name1)
Map

In [ ]:
threshold = 10
treecover_bin = treecover.gte(threshold).selfMask()
treeVisParam = {"palette": ["green"]}
Map.addLayer(treecover_bin, treeVisParam, "Tree cover bin")

In [ ]:
treeloss_year = dataset.select(["lossyear"])

treeLossVisParam = {"min": 0, "max": 21, "palette": ["yellow", "red"]}

layer_name = "Tree loss year"
Map.addLayer(treeloss_year, treeLossVisParam, layer_name)
Map.add_colorbar(treeLossVisParam, label=layer_name, layer_name=layer_name)

In [ ]:
treeloss = dataset.select(["loss"]).selfMask()
Map.addLayer(treeloss, {"palette": "red"}, "Tree loss")
Map

In [ ]:
treegain = dataset.select(["gain"]).selfMask()
Map.addLayer(treegain, {"palette": "yellow"}, "Tree gain")
Map

In [ ]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))

In [ ]:
geemap.ee_to_df(countries)

In [ ]:
style = {"color": "#ffff0088", "fillColor": "#00000000"}
Map.addLayer(countries.style(**style), {}, "Countries")

Forest area analysis

In [ ]:
geemap.zonal_stats_by_group(
    treecover_bin,
    countries,
    "forest_cover.csv",
    stat_type="SUM",
    denominator=1e6,
    scale=1000,
)

In [ ]:
geemap.pie_chart(
    "forest_cover.csv", names="NAME", values="Class_sum", max_rows=20, height=600
)

In [ ]:
geemap.bar_chart(
    "forest_cover.csv",
    x="NAME",
    y="Class_sum",
    max_rows=20,
    x_label="Country",
    y_label="Forest area (km2)",
)

Forest loss analysis.

In [ ]:
geemap.zonal_stats_by_group(
    treeloss,
    countries,
    "treeloss.csv",
    stat_type="SUM",
    denominator=1e6,
    scale=1000,
)

In [ ]:
geemap.pie_chart(
    "treeloss.csv", names="NAME", values="Class_sum", max_rows=20, height=600
)

In [ ]:
geemap.bar_chart(
    "treeloss.csv",
    x="NAME",
    y="Class_sum",
    max_rows=20,
    x_label="Country",
    y_label="Forest loss area (km2)",
)

## Surface water change analysis

### Surface water occurrence

In [ ]:
dataset = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
dataset.bandNames().getInfo()

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

image = dataset.select(["occurrence"])
region = ee.Geometry.BBox(-99.957, 46.8947, -99.278, 47.1531)

vis_params = {"min": 0.0, "max": 100.0, "palette": ["ffffff", "ffbbbb", "0000ff"]}

Map.addLayer(image, vis_params, "Occurrence")
Map.addLayer(region, {}, "ROI", True, 0.5)
Map.centerObject(region)
Map.add_colorbar(vis_params, label="Water occurrence (%)", layer_name="Occurrence")

Map

In [ ]:
hist = geemap.image_histogram(
    image,
    region,
    scale=30,
    x_label="Frequency",
    y_label="Pixel Count",
    title="Water Occurrence",
    return_df=False,
)
hist

### Surace water monthly history

In [ ]:
dataset = ee.ImageCollection("JRC/GSW1_3/MonthlyHistory")
size = dataset.size()
print(size.getInfo())

In [ ]:
# dataset.aggregate_array("system:index").getInfo()

In [ ]:
Map = geemap.Map()

image = dataset.filterDate("2020-08-01", "2020-09-01").first()
region = ee.Geometry.BBox(-99.957, 46.8947, -99.278, 47.1531)

vis_params = {"min": 0.0, "max": 2.0, "palette": ["ffffff", "fffcb8", "0905ff"]}

Map.addLayer(image, vis_params, "Water")
Map.addLayer(region, {}, "ROI", True, 0.5)
Map.centerObject(region)
Map

In [ ]:
df = geemap.jrc_hist_monthly_history(
    region=region, scale=30, frequency="month", denominator=1e4, return_df=True
)
df

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region, scale=30, frequency="month", denominator=1e4, y_label="Area (ha)"
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency="month",
    y_label="Area (ha)",
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency="month",
    y_label="Area (ha)",
    color="month",
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency="year",
    reducer="mean",
    y_label="Area (ha)",
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency="year",
    reducer="max",
    y_label="Area (ha)",
)